# Project setup

In [1]:
!pip install h2o -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# Import libraries
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML

In [3]:
# Print version numbers of the libraries as well as the environment
print(f"System platform {sys.platform}")
print(f"Python version {sys.version}")
print("-------------------------------")
print(f"Pandas version {pd.__version__}")
print(f"Numpy version {np.__version__}")
print(f"Matplotlib version {matplotlib.__version__}")
print(f"h2o version {h2o.__version__}")

System platform linux
Python version 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
-------------------------------
Pandas version 2.2.2
Numpy version 1.26.4
Matplotlib version 3.10.0
h2o version 3.46.0.6


# Data

In [4]:
df = pd.read_csv("height_prediction.csv")
df.head()

,age,gender,mean_ulna,mean_height
0,51,Male,29.0,164.3
1,53,Male,29.0,168.1
2,69,Female,24.0,161.0
3,57,Male,29.0,164.1
4,66,Male,31.0,171.0
